In [61]:
import numpy as np
import pandas as pd 
import os.path
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, svm, metrics
from sklearn.calibration import calibration_curve, CalibrationDisplay
# from sklearn.calibration import calibration_curve
import  sklearn.calibration

import matplotlib.pyplot as plt
import tensorflow_probability as tfp

from sklearn.model_selection import StratifiedKFold
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve ,KFold

In [36]:
# import sklearn
# sklearn.__version__
# sklearn.show_versions()

In [62]:
import recourse as rs
import shap
import lime

In [38]:
# import tensorflow as tf
# tensorflow.__version__

# Screening Dataset Logistic Regression

In [39]:
data_path = os.path.join("..","src","data", "screening", "screening_data.csv")
data = pd.read_csv(data_path)
training_variables = ['satisfactoryCreditHistory',
       'applicantIncomeLog', 'coapplicantIncomeLog', 'loanAmountLog',
       'loanAmountTermLog', 'totalIncomeLog', 'isMarried',
       'has1Dependent', 'has2Dependents', 'has3+Dependents', 'hasNotGraduated',
       'isSelfEmployed', 'livesSemiurbal', 'livesUrban']


In [40]:
X = data[training_variables]
y = data["outcome"]

In [41]:
def classify(model,x,y):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(x,y, test_size=0.25, random_state=47)
    model.fit(X_train, y_train)
    print('Model',model)
    print("Accuracy is", model.score(X_test, y_test)* 100)
    # cross  validation for better validation
    kfold = model_selection.KFold(n_splits=10,random_state=42, shuffle=True)
    score = model_selection.cross_val_score(model, x, y, cv=kfold)
    print("Cross Val", np.mean(score)*100)
#     ROC curve + AUC
    train_roc = metrics.plot_roc_curve(model, X_train, y_train)
    test_roc = metrics.plot_roc_curve(model, X_test, y_test) 
    plt.show()
# Reliability diagram train
    y_prob = model.predict_proba(X_train)[:, 1]
#     tf.convert_to_tensor(y_pred, dtype = tf.int64)
    prob_true, prob_pred = calibration_curve(y_train, y_prob, n_bins=10)
    disp = sklearn.calibration.CalibrationDisplay(prob_true, prob_pred, y_prob)
    disp = sklearn.calibration.CalibrationDisplay(prob_true, prob_pred, y_prob)
    disp.plot()
    
#     y_train_tensor = tf.convert_to_ten
#     y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
#     y_prob_tensor = tf.convert_to_tensor(y_prob, dtype=tf.float32)
#     y_pred_proba = clf.predict_proba(X_train)
#     logit_tensor = tf.convert_to_tensor(y_pred_proba, dtype = tf.float32, name = 'logits')
#     print(tfp.expected_calibration_error(num_bins=15, logits = logit_tensor, labels_true=y_train_tensor, labels_predicted=y_prob_tensor))
    
# reliability diagram Test
    y_prob = model.predict_proba(X_test)[:, 1]
    prob_true, prob_pred = calibration_curve(y_test, y_prob, n_bins=10)
    disp = CalibrationDisplay(prob_true, prob_pred, y_prob)
    disp = CalibrationDisplay(prob_true, prob_pred, y_prob)
    disp.plot()
#     print(tfp.python.stats.expected_calibration_error(num_bins=15, labels_true=y_test, labels_predicted=y_prob))


In [42]:
classify(LogisticRegression(), X, y)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

Model LogisticRegression()
Accuracy is 56.09756097560976
Cross Val 60.851370851370845


Function plot_roc_curve is deprecated; Function `plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: RocCurveDisplay.from_predictions or RocCurveDisplay.from_estimator.


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

# Logistic Regression - Different Approach

In [43]:
# training
data_path = os.path.join("..", "src", "data", "screening", "raw", "Train Dataset.csv")
df1 = pd.read_csv(data_path)
# missing values
df1['Credit_History']=df1['Credit_History'].fillna(df1['Credit_History'].mean())
df1['Loan_Amount_Term']=df1['Loan_Amount_Term'].fillna(df1['Loan_Amount_Term'].mean())
df1['LoanAmount']=df1['LoanAmount'].fillna(df1['LoanAmount'].mean())
df1['Dependents']=df1['Dependents'].fillna(df1['Dependents'].mode()[0])
df1['Gender']=df1['Gender'].fillna(df1['Gender'].mode()[0])
df1['Married']=df1['Married'].fillna(df1['Married'].mode()[0])
df1['Self_Employed']=df1['Self_Employed'].fillna(df1['Self_Employed'].mode()[0])
# drop loan ID
df1 = df1.drop(columns=['Loan_ID'])

In [44]:
# testing
data_path = os.path.join("..", "src", "data", "screening", "raw", "Test Dataset.csv")
data_samp = os.path.join("..", "src", "data", "screening", "raw", "Sample_Submission.csv")
df2 = pd.read_csv(data_path)
df3 = pd.read_csv(data_samp)

df2 = pd.DataFrame.merge(df2, df3, on = "Loan_ID")

# missing values
df2['Gender']=df2['Gender'].fillna(df2['Gender'].mode()[0])
df2['Dependents']=df2['Dependents'].fillna(df2['Dependents'].mode()[0])
df2['Self_Employed']=df2['Self_Employed'].fillna(df2['Self_Employed'].mode()[0])
df2['LoanAmount']=df2['LoanAmount'].fillna(df2['LoanAmount'].mean())
df2['Loan_Amount_Term']=df2['Loan_Amount_Term'].fillna(df2['Loan_Amount_Term'].mean())
df2['Credit_History']=df2['Credit_History'].fillna(df2['Credit_History'].mean())
# drop loan ID
df2=df2.drop(columns=['Loan_ID']) 

In [45]:
data=pd.concat([df1,df2],axis=0) 

In [46]:
data_path = os.path.join("..", "src", "data", "screening", "screening_data.csv")
data = pd.read_csv(data_path)
data.head()

,loanID,applicantIncome,coapplicantIncome,loanAmount,loanAmountTerm,...,hasNotGraduated,isSelfEmployed,livesSemiurbal,livesUrban,outcome
0,LP001002,5849,0.0,142.51153,360.0,...,0,0,0,1,1
1,LP001003,4583,1508.0,128.00000,360.0,...,0,0,0,0,0
2,LP001005,3000,0.0,66.00000,360.0,...,0,1,0,1,1
3,LP001006,2583,2358.0,120.00000,360.0,...,1,0,0,1,1
4,LP001008,6000,0.0,141.00000,360.0,...,0,0,0,1,1


In [47]:
class ChiSquare:
    def __init__(self, dataframe):
        self.data = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result="{0} is IMPORTANT for Prediction".format(colX)
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.data[colX].astype(str)
        Y = self.data[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)

In [48]:
#Initialize ChiSquare Class
cT = ChiSquare(data)

#Feature Selection
testColumns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']
data_cols = ['loanID', 'applicantIncome', 'coapplicantIncome', 'loanAmount',
       'loanAmountTerm', 'satisfactoryCreditHistory', 'totalIncome',
       'applicantIncomeLog', 'coapplicantIncomeLog', 'loanAmountLog',
       'loanAmountTermLog', 'totalIncomeLog', 'isMale', 'isMarried',
       'has1Dependent', 'has2Dependents', 'has3+Dependents', 'hasNotGraduated',
       'isSelfEmployed', 'livesSemiurbal', 'livesUrban']
      
       
for var in data_cols:
    cT.TestIndependence(colX=var,colY="outcome" )

loanID is NOT an important predictor. (Discard loanID from model)
applicantIncome is NOT an important predictor. (Discard applicantIncome from model)
coapplicantIncome is NOT an important predictor. (Discard coapplicantIncome from model)
loanAmount is NOT an important predictor. (Discard loanAmount from model)
loanAmountTerm is NOT an important predictor. (Discard loanAmountTerm from model)
satisfactoryCreditHistory is IMPORTANT for Prediction
totalIncome is NOT an important predictor. (Discard totalIncome from model)
applicantIncomeLog is NOT an important predictor. (Discard applicantIncomeLog from model)
coapplicantIncomeLog is NOT an important predictor. (Discard coapplicantIncomeLog from model)
loanAmountLog is NOT an important predictor. (Discard loanAmountLog from model)
loanAmountTermLog is NOT an important predictor. (Discard loanAmountTermLog from model)
totalIncomeLog is NOT an important predictor. (Discard totalIncomeLog from model)
isMale is NOT an important predictor. (Dis

In [49]:
# categorical_features = ['Gender', 'Married', 'Dependents',
#  'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']

In [50]:
# data.drop(columns=['gender', 'married', 'education','applicantIncome', 'coapplicantIncome', 'loanAmount'],inplace = True)

#Encoding Categorical Variables

# labelencoder=LabelEncoder()
# for col in data.columns:
#     data[col] = labelencoder.fit_transform(data[col])
    
#Now one hot encoding
# data=pd.get_dummies(data, columns=['Gender', 'Married', 'Dependents','Education',"Loan_Amount_Term",'Self_Employed',"Credit_History","Property_Area"],drop_first=False)
                                   
                                   

In [51]:
X = data[["satisfactoryCreditHistory", "isMarried", "livesSemiurbal", "outcome"]]
# data.columns

In [52]:
# X = data.drop(columns=['loanID', 'applicantIncome', 'coapplicantIncome', 'loanAmount','loanAmountTerm', 
#                        'totalIncome','isMale','has1Dependent', 'has2Dependents', 'has3+Dependents', 
#                        'hasNotGraduated','isSelfEmployed', 'livesSemiurbal', 'livesUrban'])

In [74]:
data.columns

Index(['loanID', 'applicantIncome', 'coapplicantIncome', 'loanAmount',
       'loanAmountTerm', 'satisfactoryCreditHistory', 'totalIncome',
       'applicantIncomeLog', 'coapplicantIncomeLog', 'loanAmountLog',
       'loanAmountTermLog', 'totalIncomeLog', 'isMale', 'isMarried',
       'has1Dependent', 'has2Dependents', 'has3+Dependents', 'hasNotGraduated',
       'isSelfEmployed', 'livesSemiurbal', 'livesUrban', 'outcome'],
      dtype='object')

## train

In [53]:
#Separating features and label
y=X['outcome']
X=X.drop(['outcome'],axis=1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2,random_state=47)

kfold = model_selection.StratifiedKFold(n_splits=3,shuffle=True, random_state=42)

In [54]:
classify(LogisticRegression(), X, y)

Model LogisticRegression()
Accuracy is 62.19512195121951
Cross Val 62.89115646258503


Function plot_roc_curve is deprecated; Function `plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: RocCurveDisplay.from_predictions or RocCurveDisplay.from_estimator.
Function plot_roc_curve is deprecated; Function `plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: RocCurveDisplay.from_predictions or RocCurveDisplay.from_estimator.


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [55]:
rss = 15
clrs = []

# clrs.append(AdaBoostClassifier(random_state=rss))
# clrs.append(GradientBoostingClassifier(random_state=rss))
# clrs.append(RandomForestClassifier(random_state=rss))
clrs.append(LogisticRegression(random_state = rss))
# clrs.append(ExtraTreesClassifier(random_state = rss))


cv_results = []
for clr in clrs :
    cv_results.append(cross_val_score(clr, X_train, y_train , scoring = 'accuracy', cv = kfold, n_jobs=-1))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
    
cv_df = pd.DataFrame({"CrossVal_Score_Means":cv_means,"CrossValerrors": cv_std,
                      "Algo":["Logistic Regression"
#                               ,"RandomForestClassifier"
#                               ,"AdaBoostClassifier"
#                               ,"Gradient Boosting"
#                               ,'ExtraTreesClassifier'
                             ]})

In [56]:
# g = sns.barplot("CrossVal_Score_Means","Algo",data = cv_df,orient = "h",**{'xerr':cv_std})
# g.set_xlabel("Mean Accuracy")
# g = g.set_title("Cross validation scores")
# print(cv_df)

In [73]:
data.columns

Index(['loanID', 'applicantIncome', 'coapplicantIncome', 'loanAmount',
       'loanAmountTerm', 'satisfactoryCreditHistory', 'totalIncome',
       'applicantIncomeLog', 'coapplicantIncomeLog', 'loanAmountLog',
       'loanAmountTermLog', 'totalIncomeLog', 'isMale', 'isMarried',
       'has1Dependent', 'has2Dependents', 'has3+Dependents', 'hasNotGraduated',
       'isSelfEmployed', 'livesSemiurbal', 'livesUrban', 'outcome'],
      dtype='object')

# Recourse

In [75]:
data

,loanID,applicantIncome,coapplicantIncome,loanAmount,loanAmountTerm,...,hasNotGraduated,isSelfEmployed,livesSemiurbal,livesUrban,outcome
0,LP001002,5849,0.0,142.51153,360.0,...,0,0,0,1,1
1,LP001003,4583,1508.0,128.00000,360.0,...,0,0,0,0,0
2,LP001005,3000,0.0,66.00000,360.0,...,0,1,0,1,1
3,LP001006,2583,2358.0,120.00000,360.0,...,1,0,0,1,1
4,LP001008,6000,0.0,141.00000,360.0,...,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
976,LP002971,4009,1777.0,113.00000,360.0,...,1,1,0,1,0
977,LP002975,4158,709.0,115.00000,360.0,...,0,0,0,1,0
978,LP002980,3250,1993.0,126.00000,360.0,...,0,0,1,0,0
979,LP002986,5000,2393.0,158.00000,360.0,...,0,0,0,0,0


In [78]:
data_path = os.path.join("..","src","data", "screening", "screening_data.csv")
data = pd.read_csv(data_path)
data.columns

Index(['loanID', 'applicantIncome', 'coapplicantIncome', 'loanAmount',
       'loanAmountTerm', 'satisfactoryCreditHistory', 'totalIncome',
       'applicantIncomeLog', 'coapplicantIncomeLog', 'loanAmountLog',
       'loanAmountTermLog', 'totalIncomeLog', 'isMale', 'isMarried',
       'has1Dependent', 'has2Dependents', 'has3+Dependents', 'hasNotGraduated',
       'isSelfEmployed', 'livesSemiurbal', 'livesUrban', 'outcome'],
      dtype='object')

In [76]:
# train a classifier

data_path = os.path.join("..","src","data", "screening", "screening_data.csv")
data = pd.read_csv(data_path)

X = data.drop(columns=['loadID', 'outcome'])
y = data['outcome']

# clf = LogisticRegression(max_iter = 1000)
# clf.fit(X, y)
# yhat = clf.predict(X)

KeyError: "['Loan_ID'] not found in axis"

In [65]:
# customize the set of actions
A = rs.ActionSet(X)  ## matrix of features. ActionSet will set bounds and step sizes by default

In [69]:
A

+---------------------------+-----------------+------------+------------+----------------+----------------+-----------+-----------+-----------+-----+-----+
|                      name |   variable type | actionable | compatible | step direction | flip direction | grid size | step type | step size |  lb |  ub |
+---------------------------+-----------------+------------+------------+----------------+----------------+-----------+-----------+-----------+-----+-----+
| satisfactoryCreditHistory | <class 'float'> |       True |        nan |              0 |            nan |       101 |  relative |      0.01 | 0.0 | 1.0 |
|                 isMarried |  <class 'bool'> |       True |        nan |              0 |            nan |         2 |  absolute |       1.0 | 0.0 | 1.0 |
|            livesSemiurbal |  <class 'bool'> |       True |        nan |              0 |            nan |         2 |  absolute |       1.0 | 0.0 | 1.0 |
+---------------------------+-----------------+------------+----

In [ ]:
# specify immutable variables
A['Married'].mutable = False

# can only specify properties for multiple variables using a list
A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].mutable = False

# education level
A['EducationLevel'].step_direction = 1  ## force conditional immutability.
A['EducationLevel'].step_size = 1  ## set step-size to a custom value.
A['EducationLevel'].step_type = "absolute"  ## force conditional immutability.
A['EducationLevel'].bounds = (0, 3)

A['TotalMonthsOverdue'].step_size = 1  ## set step-size to a custom value.
A['TotalMonthsOverdue'].step_type = "absolute"  ## discretize on absolute values of feature rather than percentile values
A['TotalMonthsOverdue'].bounds = (0, 100)  ## set bounds to a custom value.

## get model coefficients and align
A.align(clf)  ## tells `ActionSet` which directions each feature should move in to produce positive change.

# Get one individual
i = np.flatnonzero(yhat <= 0).astype(int)[0]

# build a flipset for one individual
fs = rs.Flipset(x = X.values[i], action_set = A, clf = clf)
fs.populate(enumeration_type = 'distinct_subsets', total_items = 10)
fs.to_latex()
fs.to_html()

# Run Recourse Audit on Training Data
auditor = rs.RecourseAuditor(A, coefficients = clf.coef_, intercept = clf.intercept_)
audit_df = auditor.audit(X)  ## matrix of features over which we will perform the audit.

## print mean feasibility and cost of recourse
print(audit_df['feasible'].mean())
print(audit_df['cost'].mean())
print_recourse_audit_report(X, audit_df, y)
# or produce additional information of cost of recourse by other variables
# print_recourse_audit_report(X, audit_df, y, group_by = ['y', 'Married', 'EducationLevel'])
